In [1]:
import cvxpy as cvx
import numpy as np

# First problem

To formulate the task as a network flow problem, let's consider that edges of our graph are transportations from one tile of Sk to another (${i_0 j_0}$ to ${i_1 j_1}$), their costs are euclidean distances.
Also, after moving the earth we should have flat surface with the height of 6. So, if initial height is $h_{ij}$, then inflows are $\Rightarrow b_{ij} = h_{ij} - 6$. 

$$ \min \sum_{ij} d_{ij} f_{ij}, \\
    b_j + \sum_{i \neq j} f_{ij} = \sum_{i \neq j}  f_{ji}, \\
    f_{ij} \geq 0, \\
    f_{ij} \leq h_{i}
$$
last constraint isn't required by definition of the task, however, should be always satisfied due to triangle inequality.

In [2]:
h_input = np.array([[5,5,10,10,10],
                [5,5,10,20,10],
                [0,5,5, 10, 5],
                [0,0,0,  5, 0]])

In [3]:
hxy = np.array([[h_value,i,j] for i, h_l in enumerate(h_input) for j, h_value in enumerate(h_l)])

In [4]:
x = hxy[:,1]
y = hxy[:,2]
h = hxy[:,0]

In [5]:
N = x.shape[0]

In [6]:
d = ((np.matlib.repmat(x,N,1)-np.matlib.repmat(x,N,1).transpose())**2 + \
    (np.matlib.repmat(y,N,1)-np.matlib.repmat(y,N,1).transpose())**2)**0.5

In [7]:
b = h-6

In [8]:
f = cvx.Variable(N,N)
constraints = [
    f >= 0,
    b + cvx.sum_entries(f,axis=0).T == cvx.sum_entries(f, axis=1),
    cvx.diag(f) == 0
]

In [9]:
obj = cvx.Minimize(cvx.sum_entries(cvx.mul_elemwise(d,f)))

In [10]:
prob = cvx.Problem(obj, constraints)
prob.solve()  # Returns the optimal value.
print ("status:", prob.status)
print ("optimal value %.7f" % (prob.value))
print(f.value[4])

status: optimal
optimal value 95.4910638
[[  1.26449156e-10   1.15622968e-10   1.16060300e-10   1.22334118e-10
    1.46872586e-13   1.18811323e-10   1.87744389e-10   3.20399193e-10
    2.48549084e-09   1.21461785e+00   4.46572077e-10   1.83030123e-09
    2.54372496e-09   2.60427586e-09   7.51315211e-01   4.69730400e-09
    2.66879484e-09   3.16191537e-09   9.99999976e-01   1.03406694e+00]]


# Second problem

$$
    \min_{x_{ij},y_i} \sum_{ij} {a_{ij} x_{ij} + o_i y_i },\\
    x_{ij} \in \{0,1\}, y_i \in \{0,1\} ,\\
    \sum_i x_{ij} = 1, \sum_j x_{ij} \leq c_i y_i
$$

When we have all $y_i \in \{ 0, 1 \}$, all $x_{ij}$ will be integer also. That's can be observed due to the fact that assignment_costs are random and (hopefully) never repeats for different pairs (client, f1) (client,f2). If some assignment_costs are same we can select any of (f1,f2) and objective won't change.

In [53]:
import numpy as np
import scipy as sp
from numpy import random
from numpy import matlib
import matplotlib.pyplot as plt
#from gurobipy import GRB
#import gurobipy as grb

%matplotlib inline

class Problem:
    def __init__(self, C=20, F=15):
        self.C = C
        self.F = F
        
        self.clients = np.random.rand(2, C)    # client positions
        self.facilities = np.random.rand(2, F) # facility positions

        # maximum number of clients per facility
        self.capacities = np.ones((F,), dtype=np.int32)*4;

        # assignment cost is defined as the squared distance between a client and a facility
        dx = \
            np.matlib.repmat(self.clients[0,:],F,1) - \
            np.matlib.repmat(self.facilities[0,:],C,1).transpose();
        dy = \
            np.matlib.repmat(self.clients[1,:],F,1) - \
            np.matlib.repmat(self.facilities[1,:],C,1).transpose();

        self.assignment_costs = 3*(dx*dx + dy*dy);

        self.opening_costs = np.ones((F,));
    
    def assign_random_capacities(self):
        """
        Assign more or less random capacities to facilities.
        
        This is one of the possible ways to change the problem configuration.
        In other words, use this function when testing your solution!
        """
        while True:
            self.capacities = \
                np.random.randint(2*self.C // self.F, size=self.F) + 1
            if sum(self.capacities) > self.C * 1.3:
                break
    
    def assign_random_opening_costs(self):
        """
        Assign more or less random opening costs to facilities.
        
        Same as above -- use this for your report.
        """
        # could be float, but let it be simple
        self.opening_costs = \
            np.random.randint((self.C+self.F-1) // self.F, size=self.F) + 1
    
    def plot(self, y, assignments, fig=plt):
        """
        Plot the given solution (y, assignments)
        
        Arguments:
        y, assignments -- see Problem.objective().
        fig            -- an instance of matplotlib.axes._axes.Axes to draw on.
                          Also, can be matplotlib.pyplot, in this case use the default Axes.
                          This is useful to compare your results (see "Results comparison" section).
        """
        
        y = y.astype(np.int32)
        assignments = assignments.astype(np.int32)
        
        for cli,fac in enumerate(assignments):
            fig.plot([self.clients[0,cli], self.facilities[0,fac]], \
                     [self.clients[1,cli], self.facilities[1,fac]], c=(.7,.7,.7))
            
        fig.scatter(self.clients[0,:], self.clients[1,:], s=15.0, c=assignments, \
                    vmin=0, vmax=self.F-1)
        
        fig.scatter(self.facilities[0,:], self.facilities[1,:], s=54.0, \
                    c=range(self.F), linewidth = [1*el for el in y])
        
    def objective(self, y, assignments):
        """
        Return objective function value given a solution.
        If the solution is infeasible, return infinity.
        
        Arguments:
        y           -- a binary 1D array of size F. y[i] is 1 iff i-th facility is open.
        assignments -- an integer 1D array of size C. assignments[i] is index of facility
                       that i-th client is assigned to.
        """
        assert len(y) == self.F
        assert len(assignments) == self.C
        
        y = y.astype(np.int32)
        assignments = assignments.astype(np.int32)
        
        retval = sum(is_opened*opening_cost \
                     for is_opened, opening_cost in zip(y, self.opening_costs))
        
        assignment_counts = np.zeros_like(y)
        
        for cli,fac in enumerate(assignments):
            if not y[fac]:
                return np.inf
            else:
                retval += self.assignment_costs[fac,cli]
                assignment_counts[fac] += 1
                
        if any(assignment_counts > self.capacities):
            return np.inf
            
        return retval
        
    def solve_gurobi(self, verbose=False):
        """
        Solve the problem using mixed integer program solver.
        Return `y, assignments` (see Problem.objective() docstring for format).
        
        Arguments:
        verbose -- controls Gurobi output.
        """
        m = grb.Model("facility")

        y = []
        for i_f in range(self.F):
            y.append(m.addVar(vtype=GRB.BINARY))

        x = []    
        for i_f in range(self.F):
            x.append([])
            for i_c in range(self.C):
                x[i_f].append(m.addVar(vtype=GRB.BINARY))

        # the objective is to minimize the total fixed and variable costs
        m.modelSense = GRB.MINIMIZE

        # update model to integrate new variables
        m.update()

        # set optimization objective - minimize sum of fixed costs
        obj_summands = []
        for i_f in range(self.F):
            obj_summands.append(self.opening_costs[i_f]*y[i_f])

        for i_f in range(self.F):
            for i_c in range(self.C):
                obj_summands.append(self.assignment_costs[i_f][i_c]*x[i_f][i_c])

        m.setObjective(grb.quicksum(obj_summands))

        # set constraints
        for i_c in range(self.C):
            client_constr_summands = [x[i_f][i_c] for i_f in range(self.F)]
            m.addConstr(sum(client_constr_summands), GRB.EQUAL, 1.0)

        for i_f in range(self.F):        
            facility_constr_summands = [x[i_f][i_c] for i_c in range(self.C)]
            m.addConstr(sum(facility_constr_summands), \
                        GRB.LESS_EQUAL, self.capacities[i_f]*y[i_f])       

        for i_f in range(self.F):        
            for i_c in range(self.C):
                m.addConstr(x[i_f][i_c], GRB.LESS_EQUAL, y[i_f])

        # optimize
        m.setParam(GRB.Param.OutputFlag, verbose)
        m.optimize()
        
        facilities_opened = [y[i_f].X for i_f in range(self.F)]
        clients_assignment = \
            [i_f for i_c in range(self.C) for i_f in range(self.F) if x[i_f][i_c].X != 0]
        
        return facilities_opened, clients_assignment

In [176]:
def solve_branch_and_bound(self):

    a = self.assignment_costs
    f = self.opening_costs
    x = cvx.Variable(self.F,self.C)
    y = cvx.Variable(self.F)

    obj = cvx.Minimize(cvx.sum_entries(cvx.mul_elemwise(a,x)) + cvx.sum_entries(cvx.mul_elemwise(f,y)))
    constraints_always = [
        cvx.sum_entries(x,axis=0) == 1,
        cvx.sum_entries(x,axis=1) <= cvx.mul_elemwise(self.capacities,y),
        x >= 0, x <= 1, y >= 0, y <= 1
    ]
    
    def solve_lp(node,fixed):
        constraints = constraints_always + [
            cvx.mul_elemwise(fixed[1],y) == cvx.mul_elemwise(node[1],fixed[1])
        ]
        prob = cvx.Problem(obj, constraints)
        prob.solve()
        #print(prob.status)
        return   prob.value, (x.value , y.value), fixed
    
    incumbent_value = np.inf
    incumbent = []
    node0 = ([np.zeros((self.F,self.C)),np.zeros((self.F,1))])
    fixed0 = node0.copy()
    L = [solve_lp(node0, fixed0)]
    print("initial fractional result = %f" %(L[0][0]))
    EPS = 1e-8
    while len(L) > 0:
        elem = min(L,key=lambda x: x[0])
        cobj, cnode, cfixed = elem
        L.remove(elem)
        cvariable = np.argmin(np.abs(cnode[1]-0.5))
        cfixed[1][cvariable] = 1
        children = [(cnode,cfixed), (cnode,cfixed)]
        for i in range(2):
            children[i][0][1][cvariable] = i
            chobj, chnode, chfixed = solve_lp(*children[i])
            if chobj >= incumbent_value:
                continue
            if (np.abs(np.abs(chnode[1]-0.5) - 0.5) < EPS).all() and (np.abs(np.abs(chnode[0]-0.5) - 0.5) < EPS).all(): #integer LOL
                incumbent = chnode
                incumbent_value = chobj
            else:
                L.append((chobj, chnode, chfixed))
    
    assignments = np.argmax(incumbent[0],axis=0)
    #print(assignments)
    #print(((incumbent[1]+0.5).astype(np.int64).A1))
    return ((incumbent[1]+0.5).astype(np.int64).A1), assignments.A1
    
Problem.solve_branch_and_bound = solve_branch_and_bound

In [178]:
np.random.seed(666)

problem = Problem()
problem.assign_random_capacities()
problem.assign_random_opening_costs()

print('branch & bound algorithm result = %f'%(problem.objective(*problem.solve_branch_and_bound())))

initial fractional result = 14.908308
branch & bound algorithm result = 15.037326


# Third problem

$x_{ij}$ - matrix that define if $i$-th student lives with $j$-th.
It has to be symmetric (if $i$ lives with $j$ then $j$ lives with $i$) and everybody has only one neighboor $\rightarrow$ second line of definition.
$$
    \max_{ij} A_{ij} x_{ij},\\
    \sum_i x_{ij} = 1,\\
    x = x^\top, \\
    x \geq 0, x \leq 1
$$

To add new cutting planes, we notice that in each group of 3 students only 1 pair might live together, in each group of 4 only 2 pairs, in 5 again no more than 2 pairs and so on. 
On each step we can try to find a group of three students, where are sum of $x_{ij}$ is maximum. (similar tactics for groups of 4 and so one)

In [273]:
def solveLPCP(A):
    N = A.shape[0]
    x = cvx.Variable(N,N)
    EPS = 1e-5
    constraints_always = [
        cvx.sum_entries(x, axis=0) == 1,
        x == x.T,
        x >= 0, x <= 1
    ]
    def solveLP(C):
        obj = cvx.Maximize(cvx.sum_entries(cvx.mul_elemwise(A,x)))
        prob = cvx.Problem(obj, constraints_always + C)
        prob.solve()
        return x.value
    C_used = []
    def findCP(y,C_used):
        bests = (-1,0,0)
        max_result = 0
        for y1 in range(N):
            for y2 in range(y1+1,N):
                for y3 in range(y2+1,N):
                    #print(y1,y2,y3)
                    #print(A.shape)
                    #print(y.shape)
                    result123 = A[y1,y2]*y[y1,y2] + A[y1,y3]*y[y1,y3] + A[y3,y2]*y[y3,y2]
                    if result123 > max_result and not (y1,y2,y3) in C_used:
                        max_results = result123
                        bests = (y1,y2,y3)
        return bests
        
    y = solveLP([])
    C = []
    print(A.shape)
    it = 0
    while not (np.abs(np.abs(y-0.5) - 0.5) < EPS).all():
        y1,y2,y3 = findCP(y,C_used)
        if y1 < 0:
            break
        C = C +  [x[y1,y2] + x[y2,y3] + x[y3,y1] <= 1]
        y = solveLP(C)
        it += 1
        if it % 100 == 0:
            print(it)
    return y

In [274]:
A = np.load('matrix1.npy')

In [275]:
print(solveLPCP(A))

(10, 10)
100
200
300
400
500
600


KeyboardInterrupt: 